In [ ]:
import os
import numpy as np
from tensorflow.keras.applications import VGG16
from tensorflow.keras.preprocessing import image
from tensorflow.keras.applications.vgg16 import preprocess_input
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report

# --- Load pretrained VGG16 model ---
print("Loading VGG16 model...")
vgg = VGG16(weights='imagenet', include_top=False, input_shape=(224,224,3))
print("✅ Model loaded successfully")

# --- Function to extract features ---
def extract_features(img_path):
    img = image.load_img(img_path, target_size=(224, 224))
    img_data = image.img_to_array(img)
    img_data = np.expand_dims(img_data, axis=0)
    img_data = preprocess_input(img_data)
    features = vgg.predict(img_data)
    return features.flatten()

# --- Dataset folders ---
base_path = r"C:\Users\kamal\OneDrive\Desktop\DLC Mini project\Signature"
genuine_path = os.path.join(base_path, "genuine")
forged_path = os.path.join(base_path, "forged")

# --- Load all images and labels ---
X, y = [], []

print("\nExtracting features for genuine signatures...")
for file in os.listdir(genuine_path):
    if file.endswith(".png") or file.endswith(".jpg"):
        path = os.path.join(genuine_path, file)
        feat = extract_features(path)
        X.append(feat)
        y.append(1)  # 1 = Genuine

print("\nExtracting features for forged signatures...")
for file in os.listdir(forged_path):
    if file.endswith(".png") or file.endswith(".jpg"):
        path = os.path.join(forged_path, file)
        feat = extract_features(path)
        X.append(feat)
        y.append(0)  # 0 = Forged

X = np.array(X)
y = np.array(y)

print(f"\n✅ Total signatures loaded: {len(X)}")

# --- Split dataset ---
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# --- Train Logistic Regression classifier ---
print("\nTraining classifier...")
clf = LogisticRegression(max_iter=1000)
clf.fit(X_train, y_train)

# --- Test accuracy ---
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test, y_pred)
print(f"\n🎯 Model Accuracy: {acc*100:.2f}%")
print("\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Forged', 'Genuine']))

# --- Test with new image ---
print("\nTesting a single image...")
test_image_path = os.path.join(forged_path, os.listdir(forged_path)[0])
test_feat = extract_features(test_image_path)
pred = clf.predict([test_feat])[0]

if pred == 1:
    print(f"✅ {os.path.basename(test_image_path)} → Genuine Signature")
else:
    print(f"❌ {os.path.basename(test_image_path)} → Forged Signature")
